In [74]:
import pickle
import pandas as pd
import os

import torch
with torch.profiler.profile() as profiler:
        pass


import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm, trange
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch

from sklearn.preprocessing import MinMaxScaler
import math
from torch.profiler import profile, record_function, ProfilerActivity

from operator import itemgetter
import operator

In [75]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [76]:
torch.set_default_tensor_type(torch.FloatTensor)

In [77]:
torch.__version__

'1.12.1'

In [78]:
class DogInput:
    def __init__(self, dogid, raceid,stats, dog,dog_box, lstmCellh,lstmCellc, bfsp, sp) -> None:
        self.dogid= dogid
        self.raceid = raceid
        self.stats = stats.to('cuda:0')
        self.dog = dog
        self.lstmCellh = lstmCellh.float().to('cuda:0')
        self.lstmCellc = lstmCellc.float().to('cuda:0')
        self.visited = 0
        self.bfsp = bfsp
        self.sp = sp
        
        
    def lstm_i(self, lstmInput):
        (self.lstmCellh,self.lstmCellc) = lstmInput
        # self.lstmCellh=self.lstmCellh.to(device)
        # self.lstmCellc=self.lstmCellc.to(device)
        self.visited = self.visited + 1
        # if self.visited>1:
        #     print("FOUND LEAK")
        #     sasdfasd

    def nextrace(self, raceid):
        self.nextrace_id = raceid

    def prevrace(self, raceid):
        self.prevrace_id = raceid

    def lstm_o(self, lstm_o):
        # print(lstm_o[0]._version)
        (lh,lc) = lstm_o
        if self.nextrace_id==-1:
            pass
        else:
            self.dog.races[self.nextrace_id].lstm_i((lh,lc)) #((lh.detach(), lc.detach())) #DETACH

    def detach_state(self):
        self.lstmCellh = self.lstmCellh.detach()
        self.lstmCellc = self.lstmCellc.detach()
            


In [79]:
class Dog:
    def __init__(self, dogid, hidden_size, layers) -> None:
        self.dogid = dogid
        # self.raceids = raceids #possible dictionary of race id keys dog stat outs
        self.lstmcell = 0
        self.layers = layers
        self.hidden_size = hidden_size
        self.l_debug = None
        self.races = {}

    def add_races(self, raceid, racedate, stats,nextraceid, prevraceid, box, bfsp, sp):
        self.races[raceid] = DogInput(self.dogid, raceid, stats, self, box, torch.randn(self.hidden_size),torch.randn(self.hidden_size), bfsp, sp) #this is the change
        self.races[raceid].nextrace(nextraceid)
        self.races[raceid].prevrace(prevraceid)

In [80]:
class Race:
    def __init__(self, raceid,trackOHE, dist, classes):
        self.raceid = raceid
        self.race_dist = dist.to('cuda:0')
        self.race_track = trackOHE.to('cuda:0')
        self.classes =  classes.to('cuda:0')

    def add_dogs(self, dogs_list:DogInput):
        self.dog1 = dogs_list[0]
        self.dog2 = dogs_list[1]
        self.dog3 = dogs_list[2]
        self.dog4 = dogs_list[3]
        self.dog5 = dogs_list[4]
        self.dog6 = dogs_list[5]
        self.dog7 = dogs_list[6]
        self.dog8 = dogs_list[7]
        self.dogs = dogs_list

    def nn_input(self):
        input = torch.cat([x.stats for x in self.dogs], dim = 0)
        full_input = torch.cat((self.race_dist,self.race_track, input), dim=0).to(device='cuda:0')
        self.full_input = full_input
        return full_input

    def lstm_input(self):
        l_input = [(x.lstmCellh,x.lstmCellc) for x in self.dogs]
        return l_input

    def lstm_detach(self):
        [x.detach_state for x in self.dogs]

    def list_dogs(self):
        dogs_l = [x for x in self.dogs]
        return dogs_l

    def pass_lstm_output(self, lstm_h, lstm_c):
        for i,dog in enumerate(self.dogs):
            
            lh = lstm_h[i]
            lc = lstm_c[i]
            lh,lc = lh.detach(), lc.detach()
            # lh,lc = lh.clone(), lc.clone()
            dog.lstm_o((lh,lc))
            #dog.dog.l_debug = (lh,lc)
        # zipped_lstm = zip(self.dogs,lstms)
        # [x.lstm_o(y) for x,y in zipped_lstm]

In [81]:
class Races:
    def __init__(self, hidden_size, layers, batch_size = 100) -> None:
        self.racesDict = {}
        self.dogsDict = {}
        self.raceIDs = []
        self.hidden_size = hidden_size
        self.layers = layers
        self.getter = operator.itemgetter(*range(batch_size))

    def add_race(self,raceid:str, trackOHE, dist, classes):
        self.racesDict[raceid] = Race(raceid, trackOHE, dist, classes)
        self.raceIDs.append(raceid)

    def add_dog(self,dogid):
        if dogid not in self.dogsDict.keys():
            self.dogsDict[dogid] = Dog(dogid, self.hidden_size, self.layers)
        else:
            self.dogsDict[dogid] = self.dogsDict[dogid]

    def get_race_input(self, idx) -> Race:
        raceidx = operator.itemgetter(*idx)
        #raceidx  = self.getter(idx)
        race_batch_id = raceidx(self.raceIDs)

        #race_getter = operator.itemgetter(*raceidx)

        races = [self.racesDict[x] for x in race_batch_id] #Returns list of class Race
        

        # raceidx = self.raceIDs[idx]
        #input = torch.cat([x.stats for x in races.dogs.values()], dim = 0)
        #full_input = torch.cat((self.racesDict[raceidx].race_dist,self.racesDict[raceidx].race_track, input), dim=0 )
        # dogs = [x for x in self.racesDict[raceidx].dogs]
        
        return races #self.racesDict[raceidx]

    def get_race_classes(self, idx):
        raceidx = self.raceIDs[idx]
        classes = [x for x in self.raceDict[raceidx].classes]
        return classes

    def reset_all_lstm_states(self):
        for race in self.racesDict.values:
            for dog in race.dogs:
                dog.lstmCellc = torch.rand(self.hidden_size)
                dog.lstmCellh = torch.rand(self.hidden_size)



In [82]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, dropout=0.3):
        super(Net, self).__init__()
        self.lstm1 = nn.LSTMCell(input_size, hidden_size)
        self.lstm2 = nn.LSTMCell(input_size, hidden_size)
        self.lstm3 = nn.LSTMCell(input_size, hidden_size)
        self.lstm4 = nn.LSTMCell(input_size, hidden_size)
        self.lstm5 = nn.LSTMCell(input_size, hidden_size)
        self.lstm6 = nn.LSTMCell(input_size, hidden_size)
        self.lstm7 = nn.LSTMCell(input_size, hidden_size)
        self.lstm8 = nn.LSTMCell(input_size, hidden_size)
        self.rl1 = nn.ReLU()
        self.drop1 = nn.Dropout(dropout)
        self.fc2 = nn.Linear(hidden_size * 8, 64)
        self.drop2 = nn.Dropout(dropout)
        self.rl2 = nn.ReLU()
        self.fc3 = nn.Linear(64, 8)
        self.hidden_size = hidden_size

    # x represents our data
    def forward(self, race: Race):
        #x = race.nn_input().float().to('cuda:0')
        x = torch.stack([r.full_input.float() for r in race])

        #creates list of LSTM data 
        lstm_ins = [list(i) for i in zip(*[r.lstm_input() for r in race])]

        # creates list of tensors for lstm Cells
        hCell = [torch.stack([x[0] for x in y]) for y in lstm_ins]
        cCell = [torch.stack([x[1] for x in y]) for y in lstm_ins]

        (h1, c1) = self.lstm1(x, (hCell[0], cCell[0]))
        (h2, c2) = self.lstm2(x, (hCell[1], cCell[1]))
        (h3, c3) = self.lstm3(x, (hCell[2], cCell[2]))
        (h4, c4) = self.lstm4(x, (hCell[3], cCell[3]))
        (h5, c5) = self.lstm5(x, (hCell[4], cCell[4]))
        (h6, c6) = self.lstm6(x, (hCell[5], cCell[5]))
        (h7, c7) = self.lstm7(x, (hCell[6], cCell[6]))
        (h8, c8) = self.lstm8(x, (hCell[7], cCell[7]))

        lstm_list = [
            (h1, c1),
            (h2, c2),
            (h3, c3),
            (h4, c4),
            (h5, c5),
            (h6, c6),
            (h7, c7),
            (h8, c8)
        ]

        hCello = [i for i in zip(*[x[0] for x in lstm_list])]
        cCello = [i for i in zip(*[x[1] for x in lstm_list])]

        for i,r in enumerate(race):
            r.pass_lstm_output(hCello[i],cCello[i])
            #r.lstm_detach()
        xhh = torch.cat((h1,h2, h3, h4, h5, h6, h7, h8), dim=1)
        xr1 = self.rl1(xhh)
        xd1 = self.drop1(xr1)
        xh = self.fc2(xd1)
        xd2 = self.drop2(xh)
        xr2 = self.rl2(xd2)
        xf = self.fc3(xh)

        output = F.softmax(xf, dim=1)
        return output


In [83]:
def build_dataset(data, hidden_size):

    #Load in pickeled dataframe
    resultsdf = pickle.load(data)
    dog_stats_df = pd.DataFrame(resultsdf)
    dog_stats_df = dog_stats_df.fillna(-1).drop_duplicates(subset=['dogid', 'raceid'])
    dog_stats_df['stats_cuda'] = dog_stats_df.apply(lambda x: torch.tensor(x['stats']), axis =1)
    dog_stats_df['box'] = dog_stats_df['stats'].apply(lambda x: x[0])

    #Created RaceDB
    raceDB = Races(hidden_size, 1)

    #Fill in dog portion:

    dog_stats_group = dog_stats_df.sort_values(['date']).groupby(["dogid"])

    for i,j in tqdm(dog_stats_group):
        j["next_race"] = j["raceid"].shift(-1).fillna(-1)
        j["prev_race"] = j["raceid"].shift(1).fillna(-1)
        raceDB.add_dog(i)
        j.apply(lambda x: raceDB.dogsDict[i].add_races(x['raceid'], x['date'], torch.Tensor(x['stats']),x['next_race'], x['prev_race'], x['box'], x['bfSP'], x['startprice']), axis=1)

    #Fill in races portion
    softmin = nn.Softmin(dim=0)
    races_group = dog_stats_df.groupby(['raceid'])

    null_dog = Dog("nullDog", raceDB.hidden_size, raceDB.layers)
    null_dog_i = DogInput("nullDog", "-1", torch.zeros(16), null_dog,0, torch.zeros(raceDB.hidden_size), torch.zeros(raceDB.hidden_size),0,0)
    null_dog_i.nextrace(-1)
    null_dog_i.prevrace(-1)

    null_dog_list = [null_dog] * 8
    #TO FIX LATER PROPER BOX PLACEMENT #FIXED

    races_group = dog_stats_df.groupby(['raceid'])
    for i,j in tqdm(races_group):
    #Track info tensors
        dist = torch.tensor([j.dist.iloc[0]]) 
        trackOHE = torch.tensor(j.trackOHE.iloc[0])
        #margins
        empty_dog_list = [null_dog_i]*8
        empty_margin_list = [100]*8
        boxes_list = [x for x in j['box']]
        margin_list = [x for x in j["place"]]
        dog_list = [raceDB.dogsDict[x].races[i] for x in j["dogid"]]

        #adjustedMargin = [margin_list[x-1] for x in boxes_list]
        for n,x in enumerate(boxes_list):
            empty_margin_list[x-1] = margin_list[n]
            empty_dog_list[x-1] = dog_list[n]
        adjustedMargin = softmin(torch.tensor(empty_margin_list)).to('cuda:0')
        #adjusted_dog_list = [dog_list[x-1] for x in boxes_list]
        
        raceDB.add_race(i,trackOHE,dist, adjustedMargin)
        
        
        # List of Dog Input??
        raceDB.racesDict[i].add_dogs(empty_dog_list)
        raceDB.racesDict[i].nn_input()


    return raceDB



In [84]:
#Testing
def validate_model(model,raceDB,criterion, batch_size, example_ct):
    torch.autograd.set_detect_anomaly(True)
    list_t = [] 
    last = 0
    loss_val = 0 
    correct = 0
    total = 0
    with torch.no_grad():
        for i in trange(60000,70000,batch_size):   
            races_idx = range(last,last+batch_size)
            last = i
            race = raceDB.get_race_input(races_idx)
            X = race
            y = torch.stack([x.classes for x in race])
            output = model(X)
            #print(y)

            _, actual = torch.max(y.data, 1)
            _, predicted = torch.max(output.data, 1)


            #print(predicted)
            #print(actual)
            correct += (predicted == actual).sum().item()
            total += batch_size



            loss = criterion(output, y)
            loss_val += loss
            #optimizer.zero_grad()
            #newnet.zero_grad()
            #loss.backward(retain_graph=True)  
            #optimizer.step()
            #if i %5000 == 0:
            #    print(loss)
        # optimizer.step() 
        #print(loss)
    print(f"accuray: {correct/total}")
    wandb.log({"accuracy": correct/total, "loss_val": loss_val/(10_000/batch_size), "correct": correct}, step=example_ct)

In [85]:
def train_log(loss, example_ct, epoch):
    # Where the magic happens
    wandb.log({"epoch": epoch, "loss": loss}, step=example_ct)

In [105]:
def train(model, raceDB, criterion, optimizer, config=None):
    torch.autograd.set_detect_anomaly(True)

    last = 0
    batch_size = 100
    example_ct = 0  # number of examples seen
    batch_ct = 0
    for epoch in trange(config.epochs): 
        for i in range(batch_size,60000,batch_size):
            #print(i)

            batch_ct += 1   
            races_idx = range(last,last+batch_size)
            last = i
            race = raceDB.get_race_input(races_idx)
            X = race

            y = torch.stack([x.classes for x in race])
            output = model(X)
            example_ct +=  batch_size
            batch_ct += 1

            loss = criterion(output, y)
            optimizer.zero_grad()
            loss.backward(retain_graph=True)  
            #loss.backward()
            optimizer.step()
            if ((batch_ct + 1) % 25) == 0:
                    
                    train_log(loss, example_ct, epoch)

            #[r.lstm_detach for r in race]

        print(loss)
        validate_model(model,raceDB,criterion, 8, example_ct)

    return model



In [87]:
os.getcwd()
os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
dog_stats_file = open( 'dog_stats_df.npy', 'rb')
hidden_size = 64
raceDB = build_dataset(dog_stats_file, hidden_size)

100%|██████████| 72073/72073 [00:55<00:00, 1298.22it/s]


In [103]:
def model_pipeline(my_dataset=raceDB,config=None,prev_model=None, sweep=True):
    if my_dataset:
      dataset = my_dataset    
    else:
      dataset = raceDB
    # tell wandb to get started
    with wandb.init(project="new LSTM", config=config):
      # access all HPs through wandb.config, so logging matches execution!
      wandb.define_metric("loss", summary="min")
      wandb.define_metric("test_accuracy", summary="max")
      wandb.define_metric("bfprofit", summary="max")
      config = wandb.config
      pprint.pprint(config)
      pprint.pprint(config.epochs)
      print(config)

      model = Net(144,64)
      # criterion = nn.HuberLoss()
      # criterion = nn.BCEWithLogitsLoss()
      #optimizer = optim.SGD(model.parameters(), lr=0.1)
      criterion = nn.SmoothL1Loss(reduction='mean', beta=config["l1_beta"])
      # optimizer = optim.RMSprop(model.parameters())
      optimizer = optim.AdamW(model.parameters(), lr=0.0001)
      model = model.to(device)
      # optimizer = optimizer.to(device)
      print(model)

      # and use them to train the model
      train(model, dataset, criterion, optimizer, config)

      if sweep:
        raceDB.reset_all_lstm_states


      # and test its final performance
      #test(model, test_loader)

    return model

In [89]:
test_race = raceDB.get_race_input([100,101])[0]

In [90]:
test_race.classes

tensor([4.2606e-03, 1.1582e-02, 5.7661e-04, 6.3233e-01, 3.1482e-02, 8.5577e-02,
        2.3262e-01, 1.5674e-03], device='cuda:0')

In [99]:

wandb_config_static = {'hidden_size':hidden_size,'batch_size': 360, 'dropout': 0.3, 'epochs': 10, 'f1_layer_size': 256, 'f2_layer_size': 64 , 'learning_rate': 0.00001, 'loss': 'L1', 'l1_beta':0.1,  'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}


In [92]:
os.getcwd()
os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
dog_stats_file = open( 'dog_stats_df.npy', 'rb')
resultsdf = pickle.load(dog_stats_file)
dog_stats_df = pd.DataFrame(resultsdf)

In [93]:
dog_stats_df

,dogid,raceid,date,trackOHE,dist,stats,place,startprice,margin,bfSP
0,109032142,12539914,2018-06-22,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",525.0,"[1, 0.059672102206736355, 0.001303845494665117...",8.0,49.2,17.64,120.0
1,1199430051,12539914,2018-06-22,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",525.0,"[6, 0.061256388082696595, 0.001141930450706952...",7.0,53.4,15.03,440.0
2,2131520028,12539914,2018-06-22,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",525.0,"[3, 0.05918524613560071, 0.0009662708972431771...",6.0,41.6,8.96,50.0
3,159734832,12539914,2018-06-22,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",525.0,"[7, 0.05796197139049795, 0.0010752646024627035...",5.0,2.8,7.23,3.0
4,145856635,12539914,2018-06-22,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",525.0,"[2, 0.05945284327323162, 0.0007404203490765129...",4.0,4.8,6.51,5.8
...,...,...,...,...,...,...,...,...,...,...
530017,573719159,823708139,2022-09-07,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]",600.0,"[2, 0.057828082170100215, 0.000988424427264397...",4.0,6.2,11.51,NaN
530018,564109467,823708139,2022-09-07,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]",600.0,"[5, 0.057832376285840065, 0.000857194927614034...",5.0,21.2,13.70,NaN
530019,570897048,823708139,2022-09-07,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]",600.0,"[8, 0.057757868814331315, 0.000860978290429466...",6.0,22.0,14.61,NaN
530020,496555135,823708139,2022-09-07,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]",600.0,"[7, 0.058740114629819665, 0.000743159508124979...",3.0,45.2,11.43,NaN


In [94]:
sweep_config = {"method": "random"}

metric = {"name": "profit", "goal": "maximize"}

sweep_config["metric"] = metric


parameters_dict = {
    "optimizer": {"value": "adamW"},
    "f1_layer_size": {"values": [256]},
    "f2_layer_size": {"values": [64]},
    "dropout": {"values": [0.3, 0.4, 0.5]},
    "len_data": {"value": 70000},
}

sweep_config["parameters"] = parameters_dict

parameters_dict.update(
    {
        "epochs": {"values": [100]},
        "validation_split": {"value": 0.1},
        "loss": {
            "values": [ "L1"],
            # "values": ["Huber", "MSE", "L1", "BCE", "Custom", "KL"]
            # 'value': 'l1_custom'
        },
        "num_layers": {"values": [2]},
    }
)

parameters_dict.update(
    {
        "learning_rate": {
            # a flat distribution between 0 and 0.1
            "distribution": "uniform",
            "min": 0.00011,
            "max": 0.001,
        },
        "l1_beta": {
            # a flat distribution between 0 and 0.1
            "distribution": "uniform",
            "min": 0,
            "max": 1,
        },
        "batch_size": {
            'values': [8,32,64,256,500,1000]
            # "values": [32, 64, 128, 360, 720]
            # 'values':[4,8,16,32,64,128,360]
        },
    }
)

import pprint

pprint.pprint(sweep_config)


sweep_config


{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'profit'},
 'parameters': {'batch_size': {'values': [8, 32, 64, 256, 500, 1000]},
                'dropout': {'values': [0.3, 0.4, 0.5]},
                'epochs': {'values': [100]},
                'f1_layer_size': {'values': [256]},
                'f2_layer_size': {'values': [64]},
                'l1_beta': {'distribution': 'uniform', 'max': 1, 'min': 0},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.001,
                                  'min': 0.00011},
                'len_data': {'value': 70000},
                'loss': {'values': ['L1']},
                'num_layers': {'values': [2]},
                'optimizer': {'value': 'adamW'},
                'validation_split': {'value': 0.1}}}


{'method': 'random',
 'metric': {'name': 'profit', 'goal': 'maximize'},
 'parameters': {'optimizer': {'value': 'adamW'},
  'f1_layer_size': {'values': [256]},
  'f2_layer_size': {'values': [64]},
  'dropout': {'values': [0.3, 0.4, 0.5]},
  'len_data': {'value': 70000},
  'epochs': {'values': [100]},
  'validation_split': {'value': 0.1},
  'loss': {'values': ['L1']},
  'num_layers': {'values': [2]},
  'learning_rate': {'distribution': 'uniform', 'min': 0.00011, 'max': 0.001},
  'l1_beta': {'distribution': 'uniform', 'min': 0, 'max': 1},
  'batch_size': {'values': [8, 32, 64, 256, 500, 1000]}}}

In [106]:
model = model_pipeline(raceDB,config=wandb_config_static)

wandb: WARNING Ignored wandb.init() arg project when running a sweep.


  0%|          | 0/100 [00:00<?, ?it/s]

{'batch_size': 8, 'dropout': 0.5, 'epochs': 100, 'f1_layer_size': 256, 'f2_layer_size': 64, 'l1_beta': 0.26694585209090327, 'learning_rate': 0.00012199504140915724, 'len_data': 70000, 'loss': 'L1', 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1, 'hidden_size': 64}
100
{'batch_size': 8, 'dropout': 0.5, 'epochs': 100, 'f1_layer_size': 256, 'f2_layer_size': 64, 'l1_beta': 0.26694585209090327, 'learning_rate': 0.00012199504140915724, 'len_data': 70000, 'loss': 'L1', 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1, 'hidden_size': 64}
Net(
  (lstm1): LSTMCell(144, 64)
  (lstm2): LSTMCell(144, 64)
  (lstm3): LSTMCell(144, 64)
  (lstm4): LSTMCell(144, 64)
  (lstm5): LSTMCell(144, 64)
  (lstm6): LSTMCell(144, 64)
  (lstm7): LSTMCell(144, 64)
  (lstm8): LSTMCell(144, 64)
  (rl1): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=512, out_features=64, bias=True)
  (drop2): Dropout(p=0.3, inplace=False)
  (rl2): ReLU()
  (fc3): Linear(in_fea

tensor(0.0640, device='cuda:0', grad_fn=<SmoothL1LossBackward0>)


  1%|          | 1/100 [00:47<1:18:39, 47.67s/it]

...

accuray: 0.2682


tensor(0.0588, device='cuda:0', grad_fn=<SmoothL1LossBackward0>)


100%|██████████| 100/100 [1:25:19<00:00, 51.20s/it]


accuray: 0.2637



accuracy,▁▃▅▆▆▆▇▇▇█▇▇▇▇▇▇█▇█▇█▇▇▇▇▇▇█▇████▇████▇▇
correct,▁▃▅▆▆▆▇▇▇█▇▇▇▇▇▇█▇█▇█▇▇▇▇▇▇█▇████▇████▇▇
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▆▅▅▇▆▄▆▅▇▆▃▅▆▁▅▃▄▇▅▅▆▁▆▁▅▅▄▃▆▂▆▃▆▄▁▂▄▂
loss_val,█▅▄▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▂▁▂▂▂▂▂▂▂▂▂▂▃▃
accuracy,0.2637
correct,2637
epoch,99
loss_val,0.0612


In [107]:
sweep_id = wandb.sweep(sweep_config, project="LSTM_sweeps")
CUDA_LAUNCH_BLOCKING=1
wandb.agent(sweep_id, function=model_pipeline, count=100)

Create sweep with ID: qbmypogc
Sweep URL: https://wandb.ai/nickojelly/LSTM_sweeps/sweeps/qbmypogc


wandb: Agent Starting Run: 90v2jrfj with config:
wandb: 	batch_size: 32
wandb: 	dropout: 0.4
wandb: 	epochs: 100
wandb: 	f1_layer_size: 256
wandb: 	f2_layer_size: 64
wandb: 	l1_beta: 0.7800449140759905
wandb: 	learning_rate: 0.000553647523272902
wandb: 	len_data: 70000
wandb: 	loss: L1
wandb: 	num_layers: 2
wandb: 	optimizer: adamW
wandb: 	validation_split: 0.1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


  0%|          | 0/100 [00:00<?, ?it/s]

{'batch_size': 32, 'dropout': 0.4, 'epochs': 100, 'f1_layer_size': 256, 'f2_layer_size': 64, 'l1_beta': 0.7800449140759905, 'learning_rate': 0.000553647523272902, 'len_data': 70000, 'loss': 'L1', 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
100
{'batch_size': 32, 'dropout': 0.4, 'epochs': 100, 'f1_layer_size': 256, 'f2_layer_size': 64, 'l1_beta': 0.7800449140759905, 'learning_rate': 0.000553647523272902, 'len_data': 70000, 'loss': 'L1', 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
Net(
  (lstm1): LSTMCell(144, 64)
  (lstm2): LSTMCell(144, 64)
  (lstm3): LSTMCell(144, 64)
  (lstm4): LSTMCell(144, 64)
  (lstm5): LSTMCell(144, 64)
  (lstm6): LSTMCell(144, 64)
  (lstm7): LSTMCell(144, 64)
  (lstm8): LSTMCell(144, 64)
  (rl1): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=512, out_features=64, bias=True)
  (drop2): Dropout(p=0.3, inplace=False)
  (rl2): ReLU()
  (fc3): Linear(in_features=64, out_features=8, bias=True)
)


tensor(0.0264, device='cuda:0', grad_fn=<SmoothL1LossBackward0>)


...

 27%|██▋       | 27/100 [15:45<42:07, 34.63s/it]

accuray: 0.2553


tensor(0.0672, device='cuda:0', grad_fn=<SmoothL1LossBackward0>)


 28%|██▊       | 28/100 [16:17<40:36, 33.84s/it]

accuray: 0.2607
